In [1]:
# biblioteker
# kjør "generate_vocab.py" før du kjører denne filen

"""
bruker tensorflow nightly:

pip3 install -q tensorflow_text_nightly --user
pip3 install -q tf-nightly --user

"""

import os
import re
import shutil
import string
import tensorflow as tf

import tensorflow_text as text

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing


In [2]:
# importerer og blander dataen
batch_size = 32
seed = 42

# splitter treningsdataen i to, en til trening og en til validering av treningen
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "./data/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed
)

raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    './data/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed
)

raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    './data/test',
    batch_size=batch_size
)


Found 7641 files belonging to 7 classes.
Using 6113 files for training.
Found 7641 files belonging to 7 classes.
Using 1528 files for validation.
Found 1000 files belonging to 7 classes.


In [3]:

# skaffer vokabularet vi har laget

bert_tokenizer_params=dict(lower_case=True)
reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]"]
bert_vocab_args = dict(
    # maksimum størrelse for vokabularet
    vocab_size = 8000 * 7,
    # Reserverte orddeler som må være med
    reserved_tokens=reserved_tokens,
    # flere argumenter
    bert_tokenizer_params=bert_tokenizer_params,
    learn_params={},
)

# lager en "tokenizer", som deler tekst opp i orddeler
tokenizer = text.BertTokenizer('vocab.txt', **bert_tokenizer_params)



In [4]:
#funksjoner som blir brukt til å voktorisere teksten
# altså gjøre dem om til tall i stedet for bokstaver
# hvert tall er IDen til en orddel fra vokabularet vårt

# alle vektorene må være like lange, så vi legger til
# "Padding" på slutten av vektoren hvis den ikke er lang nok

# IDen til padding
PAD_ID = 0
# Maksimum lengde for vektoren
# hvis vektoren er mindre, blir det lagt til padding
max_seq_len = 20

# denne funskjonen er her for å passe formatet vi skal bruke den i senere
def vectorize(text, label):
  # text er tekst inputtet, og label er hvilket språk det er på
  ids, mask, type_ids = preprocess_bert_input(text)
  return (ids, mask, type_ids), label

def preprocess_bert_input(text):
  # finner IDene til alle orddelene i inputtet
  ids = tokenize_text(text, max_seq_len)
  # lager en mask, som i dette tilfettet representerer lengden på vektoren vår
  mask = tf.cast(ids > 0, tf.int64)
  mask = tf.reshape(mask, [-1, max_seq_len])
  # lager den ferdige vektoren
  # først fyller lager vi en vektor med
  # den riktige lengden (shape) fyllt med nuller
  zeros_dims = tf.stack(tf.shape(mask))
  type_ids = tf.fill(zeros_dims, PAD_ID)
  # så setter vi inn de faktiske orddelenes IDer
  type_ids = tf.cast(type_ids, tf.int64)

  return (ids, mask, type_ids)

def tokenize_text(text, seq_len):
  # bruker "tokenizeren" vi lagde tidligere til å generere tokens som passer teksten
  tokens = tokenizer.tokenize(text)
  # tilpasser outputtet
  tokens = tokens.merge_dims(1, 2)[:, :seq_len]

  # klipper vekk slutten hvis den er lenger enn maksimum lengde
  tokens = tokens[:, :seq_len]
  # legger til padding hvis den er kortere enn maksimum lengde
  tokens = tokens.to_tensor(default_value=PAD_ID)
  pad = seq_len - tf.shape(tokens)[1]
  tokens = tf.pad(tokens, [[0, 0], [0, pad]], constant_values=PAD_ID)
  return tf.reshape(tokens, [-1, seq_len])

In [5]:
# vektoriserer hvert datasett, med funksjonen vi lagde med det spesielle formatet
train_ds = raw_train_ds.map(vectorize)
val_ds = raw_val_ds.map(vectorize)
test_ds = raw_test_ds.map(vectorize)


In [6]:
# printer en bit av dataen for å se at det fungerer
text_batch, label_batch = next(iter(raw_train_ds))
first_review, first_label = text_batch[0], label_batch[0]
print("Tekst: ", first_review)
print("Språk: ", raw_train_ds.class_names[first_label])
print("Vektorisert: ", vectorize(first_review, first_label)[0][0])


Tekst:  tf.Tensor(b'den koloniale bandet blei i r\xc3\xb8ynda brote i 1808 d\xc3\xa5 hovudstaden i det portugisiske koloniriket blei overf\xc3\xb8rt fr\xc3\xa5 lisboa til rio de janeiro etter napoleon sin invasjon av portugal', shape=(), dtype=string)
Språk:  nn
Vektorisert:  tf.Tensor(
[[ 248   24 1805 3454 2312 3539  284 1719   22   31 3519  684  245   15
   464  277   22 1712  383  471]], shape=(1, 20), dtype=int64)


In [7]:
# optimaliserer dataen (er ærlig talt ikke sikker på hva dette gjør men alle andre prosjekter gjør det)
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [8]:
# designer modellen som skal bli lært


model = tf.keras.Sequential([
  # omformer inputtet til tall som er "bedre å lære med"
  # i stedet for et heltall som ID for hver orddel, blir det til en liste
  # med nuller for hvert språk det ikke er og en 1 for det språket som er riktig
  # dette gjør at modellen ikke tror at orddeler som har tall nærme hverandre likner hverandre
  layers.Embedding(10000, 16, input_length=max_seq_len),
  # del av netverket som blir trent
  layers.Dropout(0.2),
  # passer på at dataen forstsatt har lik lengde
  layers.GlobalAveragePooling1D(),
  # del av netverket som blir trent
  layers.Dropout(0.2),
  # output (en for hvert språk, som det er 7 av)
  layers.Dense(7)])

# printer ut modellen
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 16)            160000    
_________________________________________________________________
dropout (Dropout)            (None, 20, 16)            0         
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 7)                 119       
Total params: 160,119
Trainable params: 160,119
Non-trainable params: 0
_________________________________________________________________


In [9]:
# kompilerer modellen
# optimisereren er en funksjon som proøver å forbedre modellen
# i hver iterasjon av læringen
optimizer = tf.keras.optimizers.Adam()

model.compile(
    # loss funksjonen beregner hvor langt unna det riktige svaret modellen er
    loss=losses.SparseCategoricalCrossentropy(from_logits=True), 
    optimizer=optimizer, metrics = ["accuracy"]
)


In [10]:
epochs = 50
# trener modellen på datasettet, 50 ganger
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/50
192/192 [==============================] - 15s 76ms/step - loss: 1.9080 - accuracy: 0.3957 - val_loss: 1.7154 - val_accuracy: 0.6826
Epoch 2/50
192/192 [==============================] - 1s 3ms/step - loss: 1.6139 - accuracy: 0.6886 - val_loss: 1.2976 - val_accuracy: 0.7801
Epoch 3/50
192/192 [==============================] - 0s 3ms/step - loss: 1.2026 - accuracy: 0.7815 - val_loss: 0.9615 - val_accuracy: 0.8377
Epoch 4/50
192/192 [==============================] - 1s 3ms/step - loss: 0.9047 - accuracy: 0.8327 - val_loss: 0.7524 - val_accuracy: 0.8639
Epoch 5/50
192/192 [==============================] - 1s 3ms/step - loss: 0.7132 - accuracy: 0.8544 - val_loss: 0.6163 - val_accuracy: 0.8789
Epoch 6/50
192/192 [==============================] - 1s 3ms/step - loss: 0.5905 - accuracy: 0.8748 - val_loss: 0.5214 - val_accuracy: 0.8940
Epoch 7/50
192/192 [==============================] - 1s 3ms/step - loss: 0.4997 - accuracy: 0.8932 - val_loss: 0.4515 - val_accuracy: 0.9090
Epoc

In [11]:
# tester hvor bra modellen er med test datasettet
loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)


32/32 [==============================] - 2s 45ms/step - loss: 0.1877 - accuracy: 0.9320
Loss:  0.18773253262043
Accuracy:  0.9319999814033508


In [12]:

# eksporterer den ferdigtrente modellen så vi kan sette inn våre egne input
export_model = tf.keras.Sequential([
  model,
  layers.Activation('sigmoid')
])

export_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=False), optimizer="adam", metrics=['accuracy']
)


In [3]:
# nå kan vi teste egne input!
inp = input("> ")
# deler inputtet inn i orddeler
inp = preprocess_bert_input(inp)
# kjører modellen på inputtet
result = export_model.predict([inp])[0]

# printer resultatet
best_index = 0
best_score = 0
for i in range(len(result)):
    print(f"{raw_train_ds.class_names[i]}: {result[i]}")
    if result[i] > best_score:
        best_score = result[i]
        best_index = i
print("Result: " + raw_train_ds.class_names[best_index])

NameError: name 'preprocess_bert_input' is not defined

In [30]:
export_model.save('sprakmodell.h5')

NameError: name 'raw_train_ds' is not defined